In [79]:
import requests
import os
import json
import time
import json
import pandas as pd

from typing import Dict, List
from requests.auth import HTTPBasicAuth
from datetime import datetime
from bs4 import BeautifulSoup
from dotenv import load_dotenv, find_dotenv

from sklearn.feature_extraction.text import TfidfVectorizer
import re
import spacy

In [ ]:
class UdemyAPI:
    def __init__(self) -> None:
        # find .env automagically by walking up directories until it's found
        dotenv_path = find_dotenv()

        # load up the entries as environment variables
        load_dotenv(dotenv_path)

        self.CLIENT_ID = os.environ.get("UDEMY_CLIENT_ID")
        self.CLIENT_SECRET = os.environ.get("UDEMY_CLIENT_SECRET")
        self.session = requests.Session()

    
    def request_data(self, url):
        try:
            response = self.session.get(url, auth=HTTPBasicAuth(self.CLIENT_ID, self.CLIENT_SECRET))

            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as errh:
            print(errh)
        except requests.exceptions.ConnectionError as errc:
            print(errc)
        except requests.exceptions.Timeout as errt:
            print(errt)
        except requests.exceptions.RequestException as err:
            print(err)
    
    def request_course_data(self):
        pass
    
    def request_course_details(self):
        pass
    
    def request_course_curriculum(self):
        pass
    

# Read the course details data

In [14]:
with open('../data/raw/courses_details_2023-06-28.json', 'r') as f:
    course_details = json.load(f)

In [15]:
title = []
url = []
id = []
headline = []
description = []
primary_category = []
primary_subcategory = []

for course in course_details:
    title.append(course['title'])
    url.append(course['url'])
    id.append(course['id'])
    headline.append(course['headline'])
    description.append(course['description'])
    primary_category.append(course['primary_category']['title_cleaned'])
    primary_subcategory.append(course['primary_subcategory']['title_cleaned'])

    
d = {'title': title, 'url': url, 'id': id, 'headline': headline, 'description': description, 'primary_category': primary_category, 'primary_subcategory': primary_subcategory}
df_courses = pd.DataFrame.from_dict(d, orient='columns')

In [16]:
df_courses

,title,url,id,headline,description,primary_category,primary_subcategory
0,The Complete Python Bootcamp From Zero to Hero...,/course/complete-python-bootcamp/,567828,Learn Python like a Professional Start from t...,<p><strong>Become a Python Programmer and lear...,development,programming-languages
1,Web Design for Web Developers: Build Beautiful...,/course/web-design-secrets/,473160,Learn web design in 1 hour with 25+ simple-to-...,<p><strong><em>IMPORTANT NOTE: The material of...,development,web-development
2,Microsoft Excel - Excel from Beginner to Advanced,/course/microsoft-excel-2013-from-beginner-to-...,793796,Excel with this A-Z Microsoft Excel Course. Mi...,<p><strong>Microsoft Excel all in One Package<...,office-productivity,microsoft
3,The Complete 2023 Web Development Bootcamp,/course/the-complete-web-development-bootcamp/,1565838,Become a Full-Stack Web Developer with just ON...,<p>Welcome to the Complete Web Development Boo...,development,web-development
4,100 Days of Code: The Complete Python Pro Boot...,/course/100-days-of-code/,2776760,Master Python by building 100 projects in 100 ...,<p>Welcome to the 100 Days of Code - The Compl...,development,programming-languages
...,...,...,...,...,...,...,...
127,Life Coaching 101: Complete Guide to Becoming ...,/course/life-coaching-101-complete-guide-to-be...,2398804,"Find out why life coaching is awesome, what it...",<p><strong>+++++++++++++++++++++++++++++++++++...,personal-development,personal-transformation
128,"NodeJS - The Complete Guide (MVC, REST APIs, G...",/course/nodejs-the-complete-guide/,1879018,"Master Node JS & Deno.js, build REST APIs with...",<p><strong>Join the most comprehensive Node.js...,development,programming-languages
129,Microsoft Excel - Data Analysis with Excel Piv...,/course/data-analysis-with-excel-pivot-tables/,977438,Master Excel pivots & learn data analysis w/ a...,<p>Excel Pivot Tables are an absolutely&nbsp;<...,office-productivity,microsoft
130,Understanding TypeScript,/course/understanding-typescript/,947098,Boost your JavaScript projects with TypeScript...,"<p><strong>Join this bestselling, completely u...",development,web-development


In [134]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    clean_text = soup.get_text()
    clean_text = clean_text.replace('\xa0', ' ')
    return clean_text
df_courses['description_cleaned'] = df_courses['description'].apply(remove_html_tags)